# Création de prompts pour le nommage et la description des clusters

In [ ]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_2022.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_clusters_composition.rds")

In [ ]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_femmes     <- mean(cluster_data$female, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$langEn, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$langFr, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethn_White, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_hetero, na.rm = TRUE) * 100
  educ_mean       <- mean(cluster_data$educ, na.rm = TRUE)
  age_mean        <- mean(cluster_data$age, na.rm = TRUE) * 80
  revenu_moyen    <- mean(cluster_data$ses_income, na.rm = TRUE) * 225000

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
    "Propose un prénom et donne une brève description pour le persona ", cluster_value, " :\n",

    "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
    if (any(cluster_vars$color == "green")) {
      paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
    } else {
      "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
    },
    "\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
    if (any(cluster_vars$color == "red")) {
      paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
    } else {
      "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
    },
    "\n\nCe cluster est composé de \n", 
    "Femmes: ", round(perc_femmes, 1), "%, 
     Immigrants: ", round(perc_immigrants, 1), "%\n",
    "Âge: ", round(age_mean, 1), " ans, 
     Revenu: ", round(revenu_moyen, 1), " k$\n",
    "Langues: 
     En ", round(perc_langEn, 1), "%, 
     Fr ", round(perc_langFr, 1), "%, 
     Autres: ", round(perc_ses_languageOther, 1), "%\n",
    "Édu: ", round(educ_mean, 1), " (0=bas, 1=élevé), 
     Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
    "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n"
  ))

  # Stocker le prompt dans la liste
  prompts[[cluster_value]] <- prompt_description
}

In [ ]:
prompts

In [ ]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [ ]:
reponses_openai

In [ ]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [ ]:
description_clusters

In [ ]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_clusters_description.rds")